# Notebook 1 - Exploration and clean up of APPL options (2021-2023)

In this Notebook:
- Loading the raw dataset
- Cleaning columns
- Filtering out anomalous rows
- Creating a clean dataframe for Machine Learning

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",50) 
pd.set_option("display.width",120)

In [14]:
# Raw file path
file_path = "../data/raw/aapl_2021_2023.csv"

# Low memory to avoid dtype warning
df_raw = pd.read_csv(file_path, nrows=600000 ,low_memory=False) 
df_raw.shape,df_raw.head()

((548163, 33),
    [QUOTE_UNIXTIME]   [QUOTE_READTIME]  [QUOTE_DATE]   [QUOTE_TIME_HOURS]   [UNDERLYING_LAST]  [EXPIRE_DATE]  \
 0        1656705600   2022-07-01 16:00    2022-07-01                 16.0               138.9     2022-07-01   
 1        1656705600   2022-07-01 16:00    2022-07-01                 16.0               138.9     2022-07-01   
 2        1656705600   2022-07-01 16:00    2022-07-01                 16.0               138.9     2022-07-01   
 3        1656705600   2022-07-01 16:00    2022-07-01                 16.0               138.9     2022-07-01   
 4        1656705600   2022-07-01 16:00    2022-07-01                 16.0               138.9     2022-07-01   
 
     [EXPIRE_UNIX]   [DTE]   [C_DELTA]   [C_GAMMA]   [C_VEGA]   [C_THETA]   [C_RHO]     [C_IV]  [C_VOLUME]    [C_LAST]  \
 0      1656705600     0.0     1.00000     0.00000    0.00000     0.00000   0.00000                           0.000000   
 1      1656705600     0.0     1.00000     0.00000    0.00000

In [15]:
df=df_raw.copy()

df.columns = (
    df.columns
    .str.strip() # Remove leading/trailing whitespace
    .str.strip("[]") # Remove brackets
)

df.columns

Index(['QUOTE_UNIXTIME', 'QUOTE_READTIME', 'QUOTE_DATE', 'QUOTE_TIME_HOURS', 'UNDERLYING_LAST', 'EXPIRE_DATE',
       'EXPIRE_UNIX', 'DTE', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_IV', 'C_VOLUME', 'C_LAST',
       'C_SIZE', 'C_BID', 'C_ASK', 'STRIKE', 'P_BID', 'P_ASK', 'P_SIZE', 'P_LAST', 'P_DELTA', 'P_GAMMA', 'P_VEGA',
       'P_THETA', 'P_RHO', 'P_IV', 'P_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT'],
      dtype='object')

In [16]:
# Conversion of date columns to datetime
df["QUOTE_DATE"] =  pd.to_datetime(df["QUOTE_DATE"])
df["EXPIRE_DATE"] =  pd.to_datetime(df["EXPIRE_DATE"])

df.dtypes.head(15)

QUOTE_UNIXTIME               int64
QUOTE_READTIME              object
QUOTE_DATE          datetime64[ns]
QUOTE_TIME_HOURS           float64
UNDERLYING_LAST            float64
EXPIRE_DATE         datetime64[ns]
EXPIRE_UNIX                  int64
DTE                        float64
C_DELTA                    float64
C_GAMMA                    float64
C_VEGA                     float64
C_THETA                    float64
C_RHO                      float64
C_IV                        object
C_VOLUME                    object
dtype: object

In [17]:
# Summary statistics of selected columns
df[["UNDERLYING_LAST", "STRIKE", "DTE", "C_BID", "C_ASK", "C_LAST", "C_IV", "C_VOLUME"]].describe().T

,count,mean,std,min,25%,50%,75%,max
UNDERLYING_LAST,548163.0,147.400698,14.963864,116.41,135.20,147.33,157.28,181.96
STRIKE,548163.0,144.655263,61.805457,18.75,100.00,140.00,185.00,320.00
DTE,548163.0,197.930478,227.047422,0.00,24.04,101.00,304.96,1059.00


In [18]:
# List of columns to convert to numeric
numeric_cols = [
    "UNDERLYING_LAST", "STRIKE", "DTE",
    "C_LAST", "C_BID", "C_ASK",
    "C_IV", "C_DELTA", "C_GAMMA", "C_VEGA",
    "C_THETA", "C_RHO",
    "STRIKE_DISTANCE", "STRIKE_DISTANCE_PCT",
    "C_VOLUME"
]

# Convert specified columns to numeric, coercing errors to NaN
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna(subset=["UNDERLYING_LAST", "STRIKE", "DTE", "C_LAST"])

In [19]:
df_clean = df.copy()

# filtering: non negative and non zero values
mask = (
    (df_clean["DTE"] > 0) &     # remove options that have already expired
    (df_clean["UNDERLYING_LAST"] > 0) &     # Positive underlying price
    (df_clean["STRIKE"] > 0) &
    (df_clean["C_LAST"] > 0) &
    (df_clean["C_BID"] >= 0) &
    (df_clean["C_ASK"] >= 0) &
    (df_clean["C_IV"] > 0) & (df_clean["C_IV"] < 5)  # Reasonable implicite volatility range
)

df_clean = df_clean[mask].copy()
df_clean.shape

(477267, 33)

In [20]:
# Mid theoretical price
df_clean["C_MID"] = (df_clean["C_BID"] + df_clean["C_ASK"]) / 2

# If C_MID = 0 (eg. bid=ask=0), fallback on C_LAST
df_clean.loc[df_clean["C_MID"] <= 0, "C_MID"] = df_clean.loc[df_clean["C_MID"] <= 0, "C_LAST"]

# Choosing C_MID comme cible principale
df_clean["TARGET_PRICE"] = df_clean["C_MID"]

df_clean[["C_BID", "C_ASK", "C_LAST", "C_MID", "TARGET_PRICE"]].head()


,C_BID,C_ASK,C_LAST,C_MID,TARGET_PRICE
75,53.70,54.10,57.20,53.900,53.900
78,38.75,39.10,33.60,38.925,38.925
79,33.75,34.15,30.65,33.950,33.950
81,23.75,24.10,24.50,23.925,23.925
82,21.75,22.10,19.95,21.925,21.925


In [21]:
# Time to expiration in years (assuming 252 trading days per year)
df_clean["T_years"] = df_clean["DTE"] / 252

# Moneyness (S / K) et log-moneyness
df_clean["S"] = df_clean["UNDERLYING_LAST"]
df_clean["K"] = df_clean["STRIKE"]
df_clean["moneyness"] = df_clean["S"] / df_clean["K"]
df_clean["log_moneyness"] = np.log(df_clean["moneyness"])


In [22]:
cols_keep = [
    "QUOTE_DATE",
    "EXPIRE_DATE",
    "DTE",
    "T_years",
    "S",
    "K",
    "moneyness",
    "log_moneyness",
    "C_IV",
    "C_DELTA",
    "C_GAMMA",
    "C_VEGA",
    "C_THETA",
    "C_RHO",
    "C_VOLUME",
    "STRIKE_DISTANCE",
    "STRIKE_DISTANCE_PCT",
    "TARGET_PRICE",
]

df_ml = df_clean[cols_keep].copy()

df_ml.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,T_years,S,K,moneyness,log_moneyness,C_IV,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,TARGET_PRICE
75,2022-07-01,2022-07-08,7.0,0.027778,138.9,85.0,1.634118,0.491103,0.00048,1.00000,0.00000,0.15666,0.00000,0.00000,0.0,53.9,0.388,53.900
78,2022-07-01,2022-07-08,7.0,0.027778,138.9,100.0,1.389000,0.328584,0.00628,1.00000,0.00000,0.00000,-0.00343,0.02150,0.0,38.9,0.280,38.925
79,2022-07-01,2022-07-08,7.0,0.027778,138.9,105.0,1.322857,0.279794,0.75157,0.99524,0.00082,0.00284,-0.01391,0.02256,0.0,33.9,0.244,33.950
81,2022-07-01,2022-07-08,7.0,0.027778,138.9,115.0,1.207826,0.188822,0.00037,1.00000,0.00000,0.34662,0.00000,0.00000,0.0,23.9,0.172,23.925
82,2022-07-01,2022-07-08,7.0,0.027778,138.9,117.0,1.187179,0.171580,0.00093,1.00000,0.00000,0.35924,0.00000,0.00000,1.0,21.9,0.158,21.925


In [23]:
checks = {
    "DTE_positive": (df_ml["DTE"] > 0),
    "S_positive": (df_ml["S"] > 0),
    "K_positive": (df_ml["K"] > 0),
    "T_years_positive": (df_ml["T_years"] > 0),
    "target_positive": (df_ml["TARGET_PRICE"] > 0)
}
check_report = {name: cond.mean() for name, cond in checks.items()}
check_report


{'DTE_positive': np.float64(1.0),
 'S_positive': np.float64(1.0),
 'K_positive': np.float64(1.0),
 'T_years_positive': np.float64(1.0),
 'target_positive': np.float64(1.0)}

In [24]:
import os

os.makedirs("data/processed", exist_ok=True)

output_path = "data/processed/aapl_calls_ml_dataset.csv"
df_ml.to_csv(output_path, index=False)

output_path

'data/processed/aapl_calls_ml_dataset.csv'